<a href="https://colab.research.google.com/github/ikanx101/AppSheet-Sales/blob/main/Converter_Jevon_Req.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _Converter_ Data Subdistributor

_Converter_ ini dibuat untuk mengubah data _output_ dari subdistributor ke bentuk tabular yang lebih _excel-user-friendly_.

__Cara pakai:__

1. Klik tombol `Connect` pada pojok kanan atas untuk menyalakan _server cloud Google Colab_.
1. _Upload file_ yang dibutuhkan, yakni:
    - Data subdistributor dengan format yang disarankan adalah `.xlsx`.
1. Klik `Runtime` >> `Run all`.
1. Tunggu hingga proses selesai (sampai muncul pesan `PROSES SELESAI`). Kira-kira akan memakan waktu sekitar 5 menit.
1. Silakan _download file_ hasil konversi.

__Keterangan__: Nama _file_ yang di-_upload_ bebas tidak ditentukan tapi disarankan satu _file_ hanya berisi **satu _sheet_ saja**.

Catatan: Karena yang kita gunakan saat ini adalah _server_ gratisnya Google, jadi kita harus selalu _install_ beberapa _packages_ setiap kali _run_ skrip ini. Konsekuensinya _runtime_-nya lebih lama.

_Last Update_: 5 November 2025


In [1]:
# @title
if(!require(readxl)){
  install.packages("readxl")
  library(readxl)
}
if(!require(janitor)){
  install.packages("janitor")
  library(janitor)
}
if(!require(janitor)){
  install.packages("tidyr")
  library(tidyr)
}
if(!require(openxlsx)){
  install.packages("openxlsx")
  library(openxlsx)
}
if(!require(stringr)){
  install.packages("stringr")
  library(stringr)
}

rm(list=ls())
gc()

library(readxl)
library(dplyr)
library(tidyr)
library(janitor)


Loading required package: readxl

Loading required package: janitor

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘janitor’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Loading required package: openxlsx

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘openxlsx’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: stringr



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,941420,50.3,1454500,77.7,1454500,77.7
Vcells,1638845,12.6,8388608,64.0,7335620,56.0



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#@title Masukkan Nama Files

file <- "Lap Penjualan Subdist KERINCI (Okt 2025).xlsx" #@param {type:"string"}
df   = read_excel(file,col_names = F) %>% clean_names()

New names:
• `` -> `...1`
• `` -> `...2`
• `` -> `...3`
• `` -> `...4`
• `` -> `...5`
• `` -> `...6`
• `` -> `...7`
• `` -> `...8`
• `` -> `...9`
• `` -> `...10`
• `` -> `...11`
• `` -> `...12`
• `` -> `...13`


In [3]:
# @title
# ==============================================================================
# tahap 2
# berikutnya kita akan pilah-pilah tabelnya
# caranya dengan mengecek ada di mana saja tabelnya

# kita cek dulu tabel header ada di mana saja
marker_1 = (1:nrow(df))[which(grepl("pelanggan",df$x2,ignore.case = T))]
# karena yang akhir gak ada pasangan, kita tambahin manual
marker_1 = c(marker_1,nrow(df))

# Fungsi untuk generate angka antara dua nilai
generate_angka_antara <- function(start, end) {
  if (end - start <= 1) {
    return(NULL)
  }
  seq(from = start, to = end - 1, by = 1)
}

# Generate angka untuk setiap pasangan berurutan
hasil_generate <- list()

for (i in 1:(length(marker_1) - 1)) {
  start <- marker_1[i]
  end <- marker_1[i + 1]

  # Generate angka antara start dan end
  angka_antara <- generate_angka_antara(start, end)

  if (!is.null(angka_antara)) {
    hasil_generate[[paste0("marker[", i, "] - marker[", i + 1, "]")]] <- angka_antara
  }
}
# ==============================================================================


# ==============================================================================
# tahap 3
# kita pisahkan tabelnya
rumah_kita = list()
for(ikanx in 1:length(hasil_generate)){
  # ambil tabel
  temp = df[hasil_generate[[ikanx]],]
  # ambil nama customer
  nama_customer = temp$x2[1]
  # ambil tabel transaksi
  temp = temp %>% filter(!is.na(x4))
  # rapihin nama kolomnya
  colnames(temp) = temp[1,]
  # hapus kolom tak perlu
  temp = temp %>% clean_names() %>% filter(sat != "Sat.")
  # tambahin nama costumer
  temp = temp %>% mutate(nama_customer)

  rumah_kita[[ikanx]] = temp
}

# kita gabungin untuk dirapikan kembali
gabung_tabel = data.table::rbindlist(rumah_kita) %>% as.data.frame()
# ==============================================================================


# ==============================================================================
# tahap 4
# kita benerin nama toko dan kode toko
# trus kita rapihin tabelnya
hasil_akhir =
  gabung_tabel %>%
  mutate(nama_customer = gsub("PELANGGAN : ","",nama_customer,fixed = T)) %>%
  mutate(nama_toko = stringr::str_squish(nama_customer)) %>%
  separate(nama_customer,
           into = c("hapus","kode_toko"),
           sep = "\\(") %>%
  mutate(kode_toko = gsub("\\)","",kode_toko)) %>%
  select(-hapus) %>%
  mutate(no_faktur = NA,
         bln_faktur = NA,
         tgl_faktur = NA,
         alamat_toko = NA,
         nama_salesman = NA,
         jenis_faktur = "PENJUALAN",
         hrg_satuan = NA,
         disc_total = NA) %>%
  mutate(brand = case_when(
    grepl("NS|nutri|sari",nama,ignore.case = T) ~ "NS",
    grepl("hilo|hi lo|hl",nama,ignore.case = T) ~ "HiLo",
    grepl("ts|tropica",nama,ignore.case = T) ~ "TS",
    grepl("lmen|l men|l-men",nama,ignore.case = T) ~ "LMen"
  )) %>%
  rename(kode_item = kode,
         nama_item = nama,
         qty_barang = gross,
         satuan = sat,
         disc_satuan = diskon,
         total_net = total_tagihan) %>%
  select(no_faktur,bln_faktur,tgl_faktur,kode_toko,nama_toko,alamat_toko,
         nama_salesman,kode_item,nama_item,qty_barang,satuan,
         hrg_satuan,disc_satuan,disc_total,total_net,brand,jenis_faktur)

openxlsx::write.xlsx(hasil_akhir,file = "converted file.xlsx")


# ==============================================================================
print("PROSES SELESAI")
# ==============================================================================

[1] "PROSES SELESAI"
